This is my first notebook for OSIC Fibrosis Progression challenge. Here I have done the EDA for both the tabular and image data.

References : https://www.kaggle.com/jagdmir/insightful-eda-on-meta-data-dicom-files

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10,8)
import os
import glob
import pydicom

# Load Datasets

In [ ]:
train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
sub = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')

train_dir = '../input/osic-pulmonary-fibrosis-progression/train'
test_dir = '../input/osic-pulmonary-fibrosis-progression/test'

In [ ]:
#Checking dimensions of all datasets

train.shape, test.shape, sub.shape

In [ ]:
# Exploring train data
train.head()

In [ ]:
# Exploring test data
test

In [ ]:
# Explore sample submission
sub.head()

As we can see from the format of submission file, we need to predict the FVC mesasurement for each patient for each week, along with the confidence value.

In [ ]:
# Checking the number of records in sample submission for a patient
sub_patient1 = sub[sub['Patient_Week'].str.contains('ID00419637202311204720264')]
sub_patient1.info()

Each patient has 146 entries across weeks for which we need to predict the FVC. Total number of patients in 5 and the total number of submissions will be 146*5 = 730.

In [ ]:
# Checking the range of values for week 
sub_patient1.head(1), sub_patient1.tail(1)

It is clear that submission file consists of records for each patient from week -12 to 133.

In [ ]:
# Checking for null values
train.isnull().sum()

In [ ]:
# Check number of unique patients in train data
train['Patient'].nunique()

Number of unqiue patients in train dataset is 176.

In [ ]:
#Check types of features available in the dataset
train.dtypes

Excluding Patient column, we have total 5 independent features and 1 target feature, that is FVC.
Out of 5 independent features, two are categorical (Sex, SmokingStatus) and rest are numeric features (Weeks, Percent,Age).

Lets explore all these features in the next section.

# Data Exploration

 **Distribution of FVC**

In [ ]:
#plt.hist(train['FVC'])
sns.distplot(train['FVC'],hist=False,color='darkred')
plt.title('FVC Distribution')

FVC ranges from 1000 to 5000 mostly. Very few cases go beyond 5000. The peak values occur at around 2500.

**Frequency of Patient Visits**

In [ ]:
df = train.groupby('Patient').count()['Weeks'].value_counts()
df

Most of the patient have got their FVC checked 9 times.

**Categorical Variables : Sex, SmokingStatus**

In [ ]:
# Lets check number of males/females in the dataset
sizes = [len(train[train['Sex']=='Male']), len(train[train['Sex']=='Female'])]
explode = (0.1,0) #explde first slice
colors = ['blue','pink']

plt.pie(sizes, explode=explode, labels= train['Sex'].unique(), colors= colors, autopct= '%1.1f%%', 
        shadow=True, startangle=140)
plt.axis('equal')
plt.title('Pie Chart for Gender Distribution')
plt.show()

79% of the patients are male.

**Checking Smoking Status**

In [ ]:
train['SmokingStatus'].value_counts()

More than 70% of the records are of people who are either ex-smokers or currently smoke.

**Distribution of Smoking Status**

In [ ]:
sizes = [len(train[train['SmokingStatus']=='Ex-smoker']), len(train[train['SmokingStatus']=='Never smoked']), 
        len(train[train['SmokingStatus']=='Currently smokes'])]
explode = (0,0,0.1)
colors = ['blue','red','green']
plt.pie(sizes, explode=explode, colors = colors, labels= train['SmokingStatus'].unique(), autopct='%1.1f%%', 
        shadow=True, startangle=140)

plt.axis('equal')
plt.title('Pie Chart for Smoking Status distribution')
plt.show()

**Age-Wise Distribution**

In [ ]:
train['Age'].min(), train['Age'].max()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize = (16,6))

#Patient age group
ageGroupLabel = 'Below 60', '60-70', '70-80', 'Above 80'

below60 = len(train[train.Age<60])
sixty_to_seventy = len(train[(train['Age']>=60) & (train['Age']<= 70)])
seventy_to_eighty = len(train[(train['Age']>70) & (train['Age']<= 80)])
above80 = len(train[train.Age>80])

patientNumbers = [below60, sixty_to_seventy, seventy_to_eighty,above80]
explode = (0,0,0,0.1)
colors = ['green','indigo','blue','red']

#Draw the pie chart
ax1.pie(patientNumbers, explode=explode, colors= colors, labels= ageGroupLabel, autopct = '%1.2f',startangle = 90)

#Aspect ratio
ax1.axis('equal')

# Distribution plot for age
sns.distplot(train['Age'],hist= False, color='red')
plt.suptitle('Age Distribution')
plt.show()


Majority of the patients belong to 60 to 80 age bracket. Very few patients are above 80.

**Week-wise Distribution**

In [ ]:
train['Weeks'].min(), train['Weeks'].max()

In [ ]:
# Dividing weeks into bins of 10 to visualize

below10 = len(train[train['Weeks']<=10])
eleven_20 = len(train[(train['Weeks']>=11) & (train['Weeks']<= 20)])
twentyone_30 = len(train[(train['Weeks']>20) & (train['Weeks']<= 30)])
thirtyone_40 = len(train[(train['Weeks']>30) & (train['Weeks']<= 40)])
fortyone_50 = len(train[(train['Weeks']>40) & (train['Weeks']<= 50)])
fiftyone_60 = len(train[(train['Weeks']>50) & (train['Weeks']<= 60)])
sixtyone_70 = len(train[(train['Weeks']>60) & (train['Weeks']<= 70)])
seventyone_80 = len(train[(train['Weeks']>70) & (train['Weeks']<= 80)])
eightyone_90 = len(train[(train['Weeks']>80) & (train['Weeks']<= 90)])
ninetyone_100 = len(train[(train['Weeks']>90) & (train['Weeks']<= 100)])
hundredone_110 = len(train[(train['Weeks']>100) & (train['Weeks']<= 110)])
hundredten_120 = len(train[(train['Weeks']>110) & (train['Weeks']<= 120)])
above120 = len(train[train.Weeks>120])

sizes = [below10, eleven_20, twentyone_30, thirtyone_40, fortyone_50, fiftyone_60,sixtyone_70,
         seventyone_80,eightyone_90,ninetyone_100,hundredone_110,hundredten_120,above120]
labels =  ['below10','eleven_20', 'twentyone_30', 'thirtyone_40', 'fortyone_50', 'fiftyone_60','sixtyone_70',
'seventyone_80','eightyone_90',
'ninetyone_100','hundredone_110','hundredten_120','above120']

fig1, (ax1, ax2)= plt.subplots(1,2,figsize=(15, 10))
theme = plt.get_cmap('prism')
ax1.set_prop_cycle("color", [theme(1. * i / len(sizes)) for i in range(len(sizes))])
_, _ = ax1.pie(sizes, startangle=90)

ax1.axis('equal')
total = sum(sizes)
ax1.legend(
    loc='upper left',
    labels=['%s, %1.1f%%' % (
        l, (float(s) / total) * 100) for l, s in zip(labels, sizes)],
    prop={'size': 11},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

# distribution plot for Weeks
sns.distplot(train['Weeks'], hist = False, color = "indigo")
plt.suptitle("Weeks Distribution")

plt.show()


It is clear that almost 50% of the patients got their FVC checked between week -5 to 25.

**Pair Plot**

In [ ]:
# Heatmap
corrMatrix = train.corr()
mask = np.triu(corrMatrix)
sns.heatmap(corrMatrix,annot=True,cmap='coolwarm',
           linewidths=1, cbar=False, fmt='.1f',
           mask=mask)
plt.show()

We can say that FVC and Percent have a decent positive correlation. It seems FVC is more when the percent value is higher.
FVC is also correlated with Age inversly.

In [ ]:
# pair plot
sns.pairplot(train)

Above plots backs the observation that FVC has strong correlation with Percent. None of the other features show
significant correlation with FVC.

**Drawing pairplots with different colors for Sex and Smoking Status**

In [ ]:
sns.pairplot(train,hue='SmokingStatus')
plt.show()

In [ ]:
sns.pairplot(train,hue='Sex')
plt.show()

**Checking pairplot for a random sample**

In [ ]:
df_100 = train[train['Patient']==train['Patient'][100]]
sns.pairplot(df_100)
plt.show()

We can say now that our understanding is correct that FVC and Percent have decent correlation.

**Mean FVC for Males vs Females**

In [ ]:
grp_sex  = train.groupby('Sex')

#draw a plot to display mean of FVC for males and females
splot = sns.barplot(x=train['Sex'].unique(),y= grp_sex['FVC'].mean())

plt.xlabel("Sex",fontsize=30)
plt.ylabel("Mean FVC",fontsize=30)
plt.title("Mean FVC Males vs Mean FVC Femailes")
plt.show()

Mean FVC is lower for males, but that can be because we do not have enough data for females

**Mean FVC for Diffetent SmokingStatus**

In [ ]:
grp_smoking = train.groupby('SmokingStatus')

splot = sns.barplot(x= train['SmokingStatus'].unique(),y=grp_smoking['FVC'].mean())

plt.xlabel("Smoking Status",fontsize=30)
plt.ylabel("Mean FVC",fontsize=30)
plt.title("Mean FVC for different Smoking categiries")
plt.show()

Mean FVC is least for patient who are smokers currently, which makes sense. But mean FVC for ex-smokers is higher than non smokers, which seems misleading.

**Mean FVC - Week Wise**

In [ ]:
plt.figure(figsize=(12,8))

#Creating bins for weeks
train['Weeks_bins'] = pd.cut(train['Weeks'],13, duplicates='drop')

#Group the data by bins
grp_weeks = train.groupby('Weeks_bins')

#Drawing barplots
splot = sns.barplot(x=train['Weeks_bins'].unique(),y=grp_weeks['FVC'].mean())

plt.xlabel("Weeks Bins",fontsize=30)
plt.ylabel("Mean FVC",fontsize=30)
plt.title("Mean FVC for different Weeks bins")
plt.show()

There is no much variation in mean FVC for different weeks

**Mean FVC - Age Wise**

In [ ]:
#Creating bins
train['Age_bins'] = pd.cut(train['Age'],4,duplicates='drop')

#Group by Age bins
grp_Age = train.groupby('Age_bins')

#Drawing plot
splot = sns.barplot(x= train['Age_bins'].unique(),y=grp_Age['FVC'].mean())

plt.xlabel("Age Bins",fontsize=30)
plt.ylabel("Mean FVC",fontsize=30)
plt.title("Mean FVC for different Age bins")
plt.show()

Again FVC is evenly distributed for all age groups

**FVC vs Weeks Plot for Randomly Selected Patients**

In [ ]:
p_id = list(train['Patient'].sample(3))
p_id

In [ ]:
# Drawing the distribution of FVC over weeks for a randomly selected patient 1

plt.plot(train[train['Patient']==p_id[0]].Weeks, train[train['Patient']==p_id[0]].FVC,
            color='darkblue')
    
plt.xlabel('Weeks',fontsize=30)
plt.ylabel('FVC',fontsize=30)
plt.title('FVC for Patient: '+p_id[0])
plt.show()

In [ ]:
# Drawing the distribution of FVC over weeks for a randomly selected patient 2

plt.plot(train[train['Patient']==p_id[1]].Weeks, train[train['Patient']==p_id[1]].FVC,
            color='darkgreen')
    
plt.xlabel('Weeks',fontsize=30)
plt.ylabel('FVC',fontsize=30)
plt.title('FVC for Patient: '+p_id[1])
plt.show()

In [ ]:
# Drawing the distribution of FVC over weeks for a randomly selected patient 1

plt.plot(train[train['Patient']==p_id[2]].Weeks, train[train['Patient']==p_id[2]].FVC,
            color='purple')
    
plt.xlabel('Weeks',fontsize=30)
plt.ylabel('FVC',fontsize=30)
plt.title('FVC for Patient: '+p_id[2])
plt.show()

Above plots give us better understanding of the relationship between weeks and FVC. Randomly selected patients are showing overall downward trend in FVC as the weeks progress.

# DICOM Files

DICOM stands for **Digital Imaging and Communications**. It is the standard for management of medical images. These files consist of images along with patient information. 

DCM files are basically 2-D slices of a single CT scan. Both the train and test directories have sub-directories for patients. Each sub-directory has multiple dicom images for a single patient, which make the CT scan for that patient.

In [ ]:
# Checking the contents of train directory
p_sizes = [] #list of number of dcm files present for each patient

for dir in os.listdir(train_dir):
    print('Patient {} has {} scans'.format(dir, len(os.listdir(train_dir+ "/"+ dir))))
    p_sizes.append(len(os.listdir(train_dir+"/"+dir)))
    
print('----------')
print('Total number of patients {}. Total DCM files {}'.format(len(os.listdir(train_dir)),sum(p_sizes)))



We have total 176 directories, one directory per patient. Total number is dicom images is 33026

In [ ]:
#Visualizing DICOM count per patient
p = sns.color_palette()
plt.hist(p_sizes,color=p[2])
plt.xlabel('Count of DCM files')
plt.ylabel('Number of patients')
plt.title('Histogram of DICOM count per patient-Training Data')

Most of the patients have between 20 to 620 DICOM files under their sub-directories. Very few patients have more than 800 files.

In [ ]:
# Checking the contents of test directory
p_sizes = [] #list of number of dcm files present for each patient

for dir in os.listdir(test_dir):
    print('Patient {} has {} scans'.format(dir, len(os.listdir(test_dir+ "/"+ dir))))
    p_sizes.append(len(os.listdir(test_dir+"/"+dir)))
    
print('----------')
print('Total number of patients {}. Total DCM files {}'.format(len(os.listdir(test_dir)),sum(p_sizes)))

Test directory has 5 sub-folders, one for each patient. These folders have 1261 files in total.

In [ ]:
#Visualizing DICOM count per patient - Test Data
p = sns.color_palette()
plt.hist(p_sizes,color=p[4])
plt.xlabel('Count of DCM files')
plt.ylabel('Number of patients')
plt.title('Histogram of DICOM count per patient-Test Data')

Test directory has around 25 to 400 images per patient.

**Checking the Size of DICOM Images**

In [ ]:
sizes = [os.path.getsize(dcm)/1000000 for dcm in glob.glob(train_dir+"/*/*.dcm")]

print('DCM File sizes: min {:.3} MB, max {:.3} MB, avg {:.3} MB, std {:.3} MB'.format(
        np.min(sizes),np.max(sizes),np.mean(sizes),np.std(sizes)))

We can see images are in range of 0.2 MB to 3.4 MB. Average image size if 0.7 MB.

**Loading a Random DICOM Image**

DICOM files can be read and processed using pydicom package. DICOM files allow storing of metadata along with pixel values. 

In [ ]:
#read a dcm file for patient ID00368637202296470751086
dcm = '/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00368637202296470751086/270.dcm'
print('Filename: {}'.format(dcm))
dcm = pydicom.read_file(dcm)

We have read the image in the dcm variable. We can print this variable to see the information related to this image.

In [ ]:
print(dcm)

We can retrieve image as a numpy array by calling dcm.pixel_array. We can replace -2000's, which are essentially NAs with Os.

In [ ]:
#display the image read above
img = dcm.pixel_array
img[img==-2000] = 0

plt.axis('off')
plt.imshow(img)
plt.show()

plt.axis('off')
plt.imshow(-img) #invert colors with - 
plt.show()

**Lets Plot Some More Random Images**

In [ ]:
#Helper function
def dicom_to_image(filename):
    dcm = pydicom.read_file(filename)
    img = dcm.pixel_array
    img[img==-2000] == 0
    return img

In [ ]:
#Lets display 20 images at random
files = glob.glob(train_dir+"/*/*.dcm")

f,plots = plt.subplots(4,5, sharex='col', sharey='row',figsize=(10,8))

for i in range(20):
    plots[i//5, i%5].axis('off')
    plots[i//5, i%5].imshow(dicom_to_image(np.random.choice(files)), cmap=plt.cm.bone)

We can try to reconstruct the layers of body from which the images were taken, by taking a single patient and sorting the scans by Slice Location.

In [ ]:
#function to sort patient dcm images
def get_slice_location(dcm):
    return float(dcm[0x0020, 0x1041].value)

#Return a list of images for given patient in ascending order of slice location
def load_patient(patient_id):
    files = glob.glob(train_dir+'/'+patient_id+'/*.dcm')
    imgs={}
    for f in files:
        dcm = pydicom.read_file(f)
        img = dcm.pixel_array
        img[img== -2000] =0
        sl= get_slice_location(dcm)
        imgs[sl] = img
        
    
    sorted_imgs = [x[1] for x in sorted(imgs.items(), key=lambda x: x[0])]
    return sorted_imgs

In [ ]:
#display all the images for patient ID00210637202257228694086
pat = load_patient('ID00210637202257228694086')
f, plots = plt.subplots(31, 10, sharex='col', sharey='row', figsize=(10, 31))
for i in range(303):
    plots[i // 10, i % 10].axis('off')
    plots[i // 10, i % 10].imshow(pat[i], cmap=plt.cm.bone)
    

**Stakcking all Images of a Patient to Create a 3D Volume**

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
#Stack up all 2D slices to make up the 3D Volume
def load_scan(patient_name):
    
    patient_directory = sorted(os.listdir(f'../input/osic-pulmonary-fibrosis-progression/train/{patient_name}'), key=(lambda f: int(f.split('.')[0])))
    volume = np.zeros((len(patient_directory), 512, 512))

    for i, img in enumerate(patient_directory):
        img_slice = pydicom.dcmread(f'../input/osic-pulmonary-fibrosis-progression/train/{patient_name}/{img}')
        volume[i] = img_slice.pixel_array
            
    return volume


In [ ]:
patient_scan = load_scan('ID00368637202296470751086')

In [ ]:
fig = plt.figure(figsize=(8, 8))

imgs = []
for ps in patient_scan:
    img = plt.imshow(ps, animated=True, cmap=plt.cm.bone)
    plt.axis('off')
    imgs.append([img])

In [ ]:
vid = animation.ArtistAnimation(fig, imgs, interval=25, blit=False, repeat_delay=1000)

In [ ]:
# lets play the video 
HTML(vid.to_html5_video())